In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

In [3]:
class TheModelClass(nn.Module):
  def __init__(self):
    super(TheModelClass, self).__init__()
    self.conv1 = nn.Conv2d(3,6,5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16* 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
  
model = TheModelClass()

In [4]:
outputs = []
def layer1_hook(module, input_, output):
  outputs.append(output)
model.pool.register_forward_hook(layer1_hook)

In [6]:
input_ = torch.randn(1,3,32,32)
output = model(input_)

In [7]:
outputs

[tensor([[[[1.1771, 0.9289, 0.6812,  ..., 0.0000, 0.0000, 0.3955],
           [0.6873, 0.1553, 0.4814,  ..., 0.4085, 1.3813, 1.0196],
           [0.7946, 0.5795, 0.4966,  ..., 1.6606, 0.2495, 0.0426],
           ...,
           [0.2028, 0.2761, 0.8648,  ..., 0.6890, 0.0000, 0.1661],
           [0.0563, 0.6666, 0.3457,  ..., 1.0488, 0.1069, 0.0000],
           [1.2425, 1.2371, 1.0972,  ..., 0.2630, 0.3082, 0.3609]],
 
          [[0.0000, 0.8358, 0.2308,  ..., 1.1812, 0.2931, 0.8762],
           [0.4743, 0.6447, 0.4462,  ..., 0.6647, 1.1148, 1.0367],
           [0.4900, 0.6693, 0.2085,  ..., 1.2237, 0.6488, 0.1970],
           ...,
           [0.2685, 0.6989, 0.4492,  ..., 0.9724, 0.0000, 0.8927],
           [0.5458, 0.1562, 0.8680,  ..., 1.1711, 0.5533, 1.0792],
           [0.5671, 0.7783, 0.4419,  ..., 1.0245, 0.2119, 0.8290]],
 
          [[0.5470, 1.1918, 0.9441,  ..., 0.1126, 0.6971, 0.8936],
           [0.6521, 1.0938, 0.6389,  ..., 0.7046, 0.1267, 0.7084],
           [0.6834, 0.89

In [8]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## Adding self-defined layers and initialization funcion

In [11]:
from torch.nn import init

def weights_init(m):
  if isinstance(m,nn.Conv2d):
    nn.init.kaiming_normal_(m.weight.data,a=0, 
                            mode = 'fan_out',
                            nonlinearity = 'relu')
    if m.bias is not None:
      torch.nn.init.ones_(m.bias)

model.apply(weights_init)


TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [12]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


